## checklist

### 진행예정
 1) 추가필요 열: 플랫폼명, 날짜 \
 2) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 3) 실행소요시간 마지막에 매직커맨드로 체크 \
 4) import는 설치 및 실행하는 거 settings.py 등으로 세팅하기 \
 5) MySQL로 나눠서 관리할 거면, 이름, 주소, 리뷰수 등은 순환주기 하루 어떤가? \
    리뷰수가 request를 한 번 더 보내야 해서 시간을 잡아먹는다 \
    그러면 어차피 스파이더를 두 개로 쪼개야하기는 하겠다


### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) 리뷰수 추가 완료

### 추후 체크 디버깅 예상
 1) 예약만실 시 그대로 반영 or 삭제

# 함수화

In [1]:
# 사전 import & settings
import scrapy, requests
from scrapy.http import TextResponse
import datetime
import re


today = datetime.date.today()
start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
            datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
gu_code = ['2012', '2019', '2016', '2014', '2015', '2020', '2018', '2017', '2021']


def goodchoice(start_date= start_date, end_date = end_date, persons=2):
    
    
    # 칼럼이 될 빈 리스트 생성
    names, levels, scores, locations, roomtypes, prices, reviews, urls = [], [], [], [], [], [], [], []
    
    
    # 구별 url 생성
    for gu in gu_code:
        url = f'https://www.goodchoice.kr/product/search/2/{gu}?sort=HIT\
&sel_date={start_date}&sel_date2={end_date}&persons={persons}'
        req = requests.get(url)
        response = TextResponse(req.url, body=req.text, encoding='utf-8')
        links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()

        
    # 각 구 당 호텔정보 크롤링 
        for link in links:
            req = requests.get(link)
            response = TextResponse(req.url, body=req.text, encoding='utf-8')
            name = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h2/text()')[0].extract()
            level = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/span/text()')[0].extract()
            try:
                score = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/span/text()')[0].extract()
            except:
                score = ['점수없음']
            location = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/p[2]/text()')[0].extract()
            try:
                roomtype = response.xpath('//*[@id="product_filter_form"]/article/div/strong/text()')[1:].extract()
            except:
                roomtype = ['예약만실']
            price = []
            for num in range(2, 2+len(roomtype)):
                try:
                    price.append(response.xpath(f'//*[@id="product_filter_form"]/article/div[{num}]/div[3]/div/div/div/p[2]/b/text()')[0].extract())
                except:
                    price.append('다른날짜확인')

    # 리뷰수 크롤링 (link에 직접 담겨있지 않으며 link url의 hotel_id로 request url 재구성 필요)
            hotel_id = re.findall("ano\=([0-9]+)\&", link)[0]
            review_url = 'https://www.goodchoice.kr/product/get_review_non'
            params = {"page": 0, "ano": hotel_id}
            response = requests.post(review_url, params)
            review = re.findall('"count":([0-9]+)', response.text)[0]


    # 열들 간 행개수 일치시키기 (print부분은 최종에서는 지우기)
            name = [name] * len(roomtype)
            level = [level] * len(roomtype)
            score = [score] * len(roomtype)
            location = [location] * len(roomtype)
            review = [review] * len(roomtype)
            landing_url = [link] * len(roomtype)
            if len(roomtype) == len(price):
                print(name[0])
            else:
                raise Exception('개수오류')
           
        
    # 데이터 추가
            names.extend(name)
            levels.extend(level)
            scores.extend(score)
            locations.extend(location)
            roomtypes.extend(roomtype)
            prices.extend(price)  
            reviews.extend(review)
            urls.extend(landing_url)
            

    # 데이터프레임화
    df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Review': reviews, 'Location': locations,
    'RoomType': roomtypes, 'Price': prices, 'Link': urls
    })
    return df

In [2]:
test = goodchoice()
test

호텔 데님
호텔 인 나인 강남
호텔 리베라
더 디자이너스 리즈 강남 프리미어
노보텔 앰배서더 서울 강남
더 리센츠 프리미엄 강남가로수길 호텔
이비스 스타일 앰배서더 강남
머큐어 앰배서더 강남 쏘도베
호텔 유리앤
임피리얼 팰리스 서울
호텔 안테룸 서울
인터컨티넨탈 서울 코엑스
그랜드 인터컨티넨탈 서울 파르나스
파크 하얏트 서울
안다즈 서울 강남
글래드 강남 코엑스센터
오크우드 프리미어 코엑스센터
도미인 서울강남
포포인츠 바이 쉐라톤 서울 강남
라마다 서울 호텔
알로프트 서울 강남
호텔 크레센도 서울
보리 호텔
호텔뉴브
호텔 포레힐
호텔 엔트라 강남
호텔 선샤인 서울
호텔 페이토 삼성
호텔 프리마
파티오세븐 호텔
에이든 바이베스트웨스턴 청담
호텔 카푸치노
베스트웨스턴 프리미어 강남 호텔
호텔 소울하다
호텔스타 강남
라까사호텔 서울
강남 스테이호텔
신라스테이 역삼
인더시티 비즈니스
신라스테이 삼성
삼정호텔
오월호텔
L7 강남 바이 롯데
강남 패밀리 호텔
호텔 컬리넌 대치
호텔 그라모스
라비타호텔
역삼 트리아호텔
베니키아 강남 노블레스
호텔 세느
삼성 XYM 코엑스
호텔 카파스
호텔 컬리넌 개포
리치웰 호텔
역삼 프리미어호텔 XYM
호텔스타 선릉
녹스 부띠끄 호텔
JW 메리어트 호텔 서울
어반 플레이스 강남
더케이 호텔 서울
신라스테이 서초
오라카이 청계산 호텔
호텔 아르누보서초
오클라우드 호텔
호텔 페이토 강남
소설 호텔
강남 아르누보씨티
서초 IMT호텔
서초 E호텔
호텔티롤
서초 라리아호텔
라바호텔
나인스 호텔
서초 아트호텔
서초 두꺼비호텔
시그니엘 서울
호텔 파크하비오
로사나 부띠끄
파로스 호텔
롯데호텔 월드
잠실 2.4 호텔
인피니 호텔
잠실 딜라이트 호텔
신천(잠실새내) 호텔 더 캐슬
잠실(방이) 호텔 더 캐슬
가락 호텔
호텔 더 디자이너스 건대
호텔 컬리넌 건대 1호점
광진 피넛호텔
호텔포코
아모렉스 호텔
호텔 컬리넌 왕십리
천호 바고호텔
천호 호텔 더 블루
노보텔 앰배서더 동대문호텔&레지던스
호텔 그레이스리 서울
티마크 그랜드 호텔
레스케이프
세종호텔
밀

,Name,Level,Score,Review,Location,RoomType,Price,Link
0,호텔 데님,2성급,8.8,895,서울 강남구 개포동 1229-14,[반나절 호캉스-숙박불가] 체크인 시 배정 (...,"40,000원",https://www.goodchoice.kr/product/detail?ano=6...
1,호텔 데님,2성급,8.8,895,서울 강남구 개포동 1229-14,[반나절 호캉스-숙박불가] 체크인 시 배정 (...,"50,000원",https://www.goodchoice.kr/product/detail?ano=6...
2,호텔 데님,2성급,8.8,895,서울 강남구 개포동 1229-14,[반나절 호캉스-숙박불가] 비즈니스 더블 (객...,"50,000원",https://www.goodchoice.kr/product/detail?ano=6...
3,호텔 데님,2성급,8.8,895,서울 강남구 개포동 1229-14,[반나절 호캉스-숙박불가] 비즈니스 더블 (객...,"60,000원",https://www.goodchoice.kr/product/detail?ano=6...
4,호텔 데님,2성급,8.8,895,서울 강남구 개포동 1229-14,디럭스 더블,"80,000원",https://www.goodchoice.kr/product/detail?ano=6...
...,...,...,...,...,...,...,...,...
2008,신림 어반호텔,1성급,9.1,18,서울 관악구 신림동 1569-5,스위트,"120,000원",https://www.goodchoice.kr/product/detail?ano=1...
2009,신림 어반호텔,1성급,9.1,18,서울 관악구 신림동 1569-5,VIP,"140,000원",https://www.goodchoice.kr/product/detail?ano=1...
2010,신트라 호텔,1성급,9.0,129,서울 관악구 신림동 1421-47,스탠다드,"75,000원",https://www.goodchoice.kr/product/detail?ano=1...
2011,신트라 호텔,1성급,9.0,129,서울 관악구 신림동 1421-47,디럭스,"80,000원",https://www.goodchoice.kr/product/detail?ano=1...


In [4]:
print(test.shape)
test.tail(10)

(2013, 8)


,Name,Level,Score,Review,Location,RoomType,Price,Link
2003,뜨랑블루 호텔,2성급,9.1,231,서울 관악구 신림동 1433-123,스위트,"150,000원",https://www.goodchoice.kr/product/detail?ano=1...
2004,뜨랑블루 호텔,2성급,9.1,231,서울 관악구 신림동 1433-123,프리미어 스위트,"150,000원",https://www.goodchoice.kr/product/detail?ano=1...
2005,뜨랑블루 호텔,2성급,9.1,231,서울 관악구 신림동 1433-123,로얄 스위트 (수영장 제외 / 조식 2인 포함),"170,000원",https://www.goodchoice.kr/product/detail?ano=1...
2006,신림 어반호텔,1성급,9.1,18,서울 관악구 신림동 1569-5,디럭스,"70,000원",https://www.goodchoice.kr/product/detail?ano=1...
2007,신림 어반호텔,1성급,9.1,18,서울 관악구 신림동 1569-5,로얄 디럭스,"80,000원",https://www.goodchoice.kr/product/detail?ano=1...
2008,신림 어반호텔,1성급,9.1,18,서울 관악구 신림동 1569-5,스위트,"120,000원",https://www.goodchoice.kr/product/detail?ano=1...
2009,신림 어반호텔,1성급,9.1,18,서울 관악구 신림동 1569-5,VIP,"140,000원",https://www.goodchoice.kr/product/detail?ano=1...
2010,신트라 호텔,1성급,9.0,129,서울 관악구 신림동 1421-47,스탠다드,"75,000원",https://www.goodchoice.kr/product/detail?ano=1...
2011,신트라 호텔,1성급,9.0,129,서울 관악구 신림동 1421-47,디럭스,"80,000원",https://www.goodchoice.kr/product/detail?ano=1...
2012,신트라 호텔,1성급,9.0,129,서울 관악구 신림동 1421-47,비즈니스 더블룸,"85,000원",https://www.goodchoice.kr/product/detail?ano=1...


In [ ]:
# items.py

In [ ]:
# spider.py